In [ ]:
from gtts import gTTS
import speech_recognition as sr
import os
import webbrowser
import smtplib, ssl
import re
from weather import Weather

In [ ]:
def talkToMe(audio):
    "speaks audio passed as argument"
    print(audio)
    tts = gTTS(text=audio)
    tts.save('audio.mp3')
    os.system("mpg123 audio.mp3")

In [ ]:
! pip install python3-weather-api
! pip install pyaudio

In [ ]:
! pip install libportaudio2

In [ ]:
! pip install wheel

In [ ]:
def myCommand():
    "listens for commands"

    r = sr.Recognizer()

    with sr.Microphone() as source:
        print('Ready...')
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=1)
        audio = r.listen(source)

    try:
        command = r.recognize_google(audio).lower()
        print('You said: ' + command + '\n')

    #loop back to continue to listen for commands if unrecognizable speech is received
    except sr.UnknownValueError:
        print('Your last command couldn\'t be heard')
        command = myCommand();

    return command

In [ ]:
def assistant(command):
    "if statements for executing commands"

    if 'open google' in command:
        chrome_path = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
        webbrowser.register('chrome', None,webbrowser.BackgroundBrowser(chrome_path),1)
        url = 'https://www.google.com/'
        webbrowser.get('chrome').open(url)

    elif 'what\'s up' in command:
        talkToMe('Just doing my thing')
        
    elif 'current weather in' in command:
        reg_ex = re.search('current weather in (.*)', command)
        if reg_ex:
            city = reg_ex.group(1)
            weather = Weather()
            location = weather.lookup_by_location(city)
            condition = location.condition()
            talkToMe('The Current weather in %s is %s The tempeture is %.1f degree' % (city, condition.text(), (int(condition.temp())-32)/1.8))
        
    elif 'email' in command:
        talkToMe('Who is the recipient?')
        recipient = myCommand()

        if 'navneet' in recipient:
            talkToMe('What should I say?')
            content = myCommand()

            #init gmail SMTP
            mail = smtplib.SMTP_SSL('smtp.gmail.com', 587)

            #identify to server
            mail.ehlo()

            #encrypt session
            mail.starttls()

            #login
            mail.login('abc@gmail', 'Your_password')

            #send message
            mail.sendmail('Name', 'xyz@gmail.com', content)

            #end mail connection
            mail.close()

            talkToMe('Email sent.')

        else:
            talkToMe('I don\'t know what you mean!')


talkToMe('I am ready for your command')

#loop to continue executing multiple commands
while True:
    assistant(myCommand())